# Title
[]()

In [3]:

# import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
# from datetime import datetime
# sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")

# Initiate dictionaries

In [1]:
results_dict = dict()

In [5]:
contacts = load_json('contacts.json', '../src/private/')

# Iteration 1

In [8]:
import requests
from datetime import datetime
import json
import sys
from urllib.parse import urlencode
# import boto3

def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='SamLab',
        auth_token_path='private/auth_token_response.json'
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - auth_token_path (str): Local path to the JSON file containing the authentication token.
        Default value works if script is run from project root folder.If running from notebook directory, use '../src/private/auth_token_response.json'

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = f'Send message to contact {contactId}'
                payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        try:
            with open(auth_token_path) as token_file:
                token = json.load(token_file)[location]
        except:
            try:
                s3 = boto3.client('s3')
                response = s3.get_object(Bucket='ownitfit-silvhua', Key='auth_token_response.json')
                token = json.loads(response['Body'].read().decode('utf-8'))[location]
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print(f'Error in line {lineno} of {filename}: {str(error)}')
                return None

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-04-15",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }


        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'
    
iteration = 1
contactId = contacts['me']
results_dict[iteration] = ghl_request(
    contactId, 'getContact', auth_token_path='../src/private/auth_token_response.json'
    )
results_dict[iteration]

Status code 200: OK


{'contact': {'id': 'C3a796qLEF4HtDjvKJ1K',
  'country': 'AU',
  'fullNameLowerCase': 'silvia test 2023-08-16 13:35',
  'lastName': 'test 2023-08-16 13:35',
  'type': 'lead',
  'firstNameLowerCase': 'silvia',
  'createdBy': {'sourceId': 'AwjBxNxoPkiLEKKZ1Gbb',
   'channel': 'APP',
   'source': 'WEB_USER',
   'sourceName': 'silvia hua',
   'timestamp': '2023-08-16T20:35:47.598Z'},
  'locationId': 'QSZ6lvarPNxtmTFl84W8',
  'tags': [],
  'dateAdded': '2023-08-16T20:35:47.598Z',
  'additionalEmails': [],
  'attributionSource': {'sessionSource': 'CRM UI',
   'mediumId': None,
   'medium': 'manual'},
  'lastNameLowerCase': 'test 2023-08-16 13:35',
  'firstName': 'Silvia',
  'additionalPhones': [],
  'emailLowerCase': 'silvia.hua@mail.mcgill.ca',
  'email': 'silvia.hua@mail.mcgill.ca',
  'dndSettings': {'Call': {'code': '103',
    'message': "Updated by 'silvia hua' at 2023-08-16T20:38:29.868Z",
    'status': 'inactive'},
   'Email': {'code': '103',
    'message': "Updated by 'silvia hua' at 2

In [13]:
print(results_dict[iteration].keys())
print(results_dict[iteration]['contact'].keys())
results_dict[iteration]['contact']

dict_keys(['contact', 'traceId', 'status_code', 'response_reason'])
dict_keys(['id', 'country', 'fullNameLowerCase', 'lastName', 'type', 'firstNameLowerCase', 'createdBy', 'locationId', 'tags', 'dateAdded', 'additionalEmails', 'attributionSource', 'lastNameLowerCase', 'firstName', 'additionalPhones', 'emailLowerCase', 'email', 'dndSettings', 'dnd', 'phone', 'dateUpdated', 'customFields'])


{'id': 'C3a796qLEF4HtDjvKJ1K',
 'country': 'AU',
 'fullNameLowerCase': 'silvia test 2023-08-16 13:35',
 'lastName': 'test 2023-08-16 13:35',
 'type': 'lead',
 'firstNameLowerCase': 'silvia',
 'createdBy': {'sourceId': 'AwjBxNxoPkiLEKKZ1Gbb',
  'channel': 'APP',
  'source': 'WEB_USER',
  'sourceName': 'silvia hua',
  'timestamp': '2023-08-16T20:35:47.598Z'},
 'locationId': 'QSZ6lvarPNxtmTFl84W8',
 'tags': [],
 'dateAdded': '2023-08-16T20:35:47.598Z',
 'additionalEmails': [],
 'attributionSource': {'sessionSource': 'CRM UI',
  'mediumId': None,
  'medium': 'manual'},
 'lastNameLowerCase': 'test 2023-08-16 13:35',
 'firstName': 'Silvia',
 'additionalPhones': [],
 'emailLowerCase': 'silvia.hua@mail.mcgill.ca',
 'email': 'silvia.hua@mail.mcgill.ca',
 'dndSettings': {'Call': {'code': '103',
   'message': "Updated by 'silvia hua' at 2023-08-16T20:38:29.868Z",
   'status': 'inactive'},
  'Email': {'code': '103',
   'message': "Updated by 'silvia hua' at 2023-08-16T20:38:29.868Z",
   'status': 

In [14]:
results_dict[iteration]['contact']['tags']

[]

In [15]:
iteration = 1.1
contactId = contacts['me']
results_dict[iteration] = ghl_request(
    contactId, 'getContact', auth_token_path='../src/private/auth_token_response.json'
    )
results_dict[iteration]['contact']['tags']

Status code 200: OK


['"sam lab members"', '"sam lab total list"']

In [20]:
print(type(results_dict[iteration]['contact']['tags']))
print(type(results_dict[iteration]['contact']['tags'][0]))
print(type(results_dict[iteration]['contact']['tags'][0]))
print(results_dict[iteration]['contact']['tags'][0])
results_dict[iteration]['contact']['tags'][0][0]

<class 'list'>
<class 'str'>
<class 'str'>
"sam lab members"


'"'

# *End of Page*